## 1. CHECK SUBMODULES

**!!!Caveat :** This command is better run in terminal to see STDOUT clearly. Iff you opt to run the script in terminal, execute it from the <u>**module-training**</u> directory and **not** the <u>**project root directory**</u>!

<hr style="border:2px solid gray"> </hr>

In [2]:
!./scripts/check-submodules.sh

## 2. CREATE VIRTUAL ENVIRONMENT

**!!!Caveat :** This script should be run in terminal to grant install permission (installs essential [dependencies](scripts/dependencies.sh)), and should be executed from the <u>**module-training**</u> directory and **not** the <u>**project root directory**</u>! After running the command (creating the virtual environment), **change jupyter kernel** to the created virtual environment.

<hr style="border:2px solid gray"> </hr>

`./scripts/venv.sh`


## 3. INSTALL TF OBJECT DETECTION

**!!!Caveat :** This command is better run in terminal to see STDOUT clearly. After installing TF object detection,`tfod.sh` executes verification which provides feedback (pass/fail) of verification tests. Iff you opt to run the script in terminal, execute it from the <u>**module-training**</u> directory and **not** the <u>**project root directory**</u>!

<hr style="border:2px solid gray"> </hr>


In [ ]:
!./scripts/tfod.sh

## 4. GET PRETRAINED MODEL FROM [TENSOR FLOW MODEL ZOO](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

<hr style="border:2px solid gray"> </hr>

In [ ]:
!./scripts/pretrained-model.sh

## 5. START MODEL TRAINING [ TRANSFER LEARNING ]

**!!!CAVEAT :** Before proceeding from here, be sure install [tallying versions](https://www.tensorflow.org/install/source#gpu) of tensorflow, tensorflow-gpu, cuDNN, and CUDA. Also emember to link CUDA library (i.e., add *!!!correct* libcudart to path), e.g., from terminal 

######  # find shared object
```
sudo find / -name 'libcudart.so*'

# example output:
/usr/lib/x86_64-linux-gnu/libcudart.so.10.1
/usr/lib/x86_64-linux-gnu/libcudart.so
```
###### # add it to path 
```
export PATH=/usr/lib/x86_64-linux-gnu${PATH:+:${PATH}}
export LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu:${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
```
###### # set permission
```
sudo chmod a+r /usr/lib/x86_64-linux-gnu/libcuda*
```

<hr style="border:2px solid gray"> </hr>

### a.  create label map

In [2]:
import os

# path to annotations file
ann_file = os.path.join('annotations', 'annotations.pbtxt')

# labels corresonds to class names defined during image annotation
labels = [{'name':'smartphone', 'id':1}]

# write annotations file
with open(ann_file, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

### b. create TF records

In [3]:
!./scripts/tfrecords.sh

Successfully created the TFRecord file: /home/everett/Repositories/research/projects/test/TFOD-model-training/model-training/records/train.record
Successfully created the TFRecord file: /home/everett/Repositories/research/projects/test/TFOD-model-training/model-training/records/test.record


### c. copy pipeline configuration of pretrainded model (transfer learning)

In [5]:
# import TS object detection TODO: TESTING FROM HERE ONWARDS!!!
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [6]:
import os
pretrained_pipeline_config_file = os.path.join('models/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config')
models = os.path.join('models')

In [7]:
!cp {pretrained_pipeline_config_file} {models}

### check model's pipeline config after update (optional) 

In [9]:
config = config_util.get_configs_from_pipeline_file(os.path.join('models/pipeline.config'))
# config # <-- uncomment to see cloned pipeline config file

In [10]:
# configure training model's pipeline config using cloned pre-trained model's pipeline config
model_pipeline_config_file = os.path.join('models/pipeline.config')
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

# parse cloned pipeline config
with tf.io.gfile.GFile(model_pipeline_config_file, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 
    
# update model settings
model_test_record = [os.path.join('records/train.record')]
model_train_record = [os.path.join('records/train.record')]
model_checkpoint = os.path.join('models/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0')

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4

pipeline_config.train_input_reader.label_map_path= ann_file
pipeline_config.train_config.fine_tune_checkpoint = model_checkpoint
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = model_train_record

pipeline_config.eval_input_reader[0].label_map_path = ann_file
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = model_test_record

config_text = text_format.MessageToString(pipeline_config)  

with tf.io.gfile.GFile(model_pipeline_config_file, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text) 

### check model's pipeline config after update (optional) 

In [11]:
config = config_util.get_configs_from_pipeline_file(model_pipeline_config_file)
# config # <-- uncomment to see config post update

### d. train model

**!!!Caveat :** This command is better run in terminal to see STDOUT clearly. Iff you opt to run the script in terminal, execute it from the <u>**module-training**</u> directory and **not** the <u>**project root directory**</u>!

In [ ]:
!./scripts/train.sh

### e. evaluate model (optional)

**!!!Caveat :** This command is better run in terminal to see STDOUT clearly. Iff you opt to run the script in terminal, execute it from the <u>**module-training**</u> directory and **not** the <u>**project root directory**</u>!

In [ ]:
!./scripts/evaluate.sh

### f. load pipeline config and restore chekpoint

In [76]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [78]:
# load pipeline config and build model
configs = config_util.get_configs_from_pipeline_file(model_pipeline_config_file)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
model_checkpoint = os.path.join('models/ckpt-3') ## todo: dynamically find last checkpoint
ckpt.restore(model_checkpoint).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections